In [1]:
import os
import torch
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm as tq
from torchtext.data import TabularDataset

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
USE_STEMEER = False
SEED = 42
QUICK = True
TEST_SIZE = 0.2
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda:0'

In [3]:
# seeding function for reproducibility
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

In [4]:
# Reading the csv file and removing unnecessary columns
df = pd.read_csv("../input/sentiment140/training.1600000.processed.noemoticon.csv",
                 encoding="latin1",
                 header=None)
df = df.rename(columns={0:"sentiment",
                        1:"id",
                        2:"time",
                        3:"query",
                        4:"username",
                        5:"text"})
df = df[["sentiment","text"]]
df["sentiment"] = df["sentiment"].map({0: 0, 4: 1})
df.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [5]:
df.sentiment.value_counts()
# Looks like dataset is well balanced :)

1    800000
0    800000
Name: sentiment, dtype: int64

## Text Preprocessing

In [6]:
import re
from nltk.stem.porter import PorterStemmer

use_stemmer = USE_STEMEER
if use_stemmer:
      porter_stemmer = PorterStemmer()

def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|;\s?D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet


def preprocess_tweet(tweet):
    processed_tweet = []
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip " and ' from tweet
    tweet = tweet.strip('"\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    # Convert to lower case
    tweet = tweet.lower()
    
    words = tweet.split()
    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                word = str(porter_stemmer.stem(word))
        processed_tweet.append(word)
    return ' '.join(processed_tweet)

In [7]:
# Example output
print(df.text[2])
print(preprocess_tweet(df.text[2]))

@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
user_mention i dived many times for the ball managed to save 50% the rest go out of bounds


In [8]:
%%time
df['Processed_text'] = df.text.apply(preprocess_tweet)

CPU times: user 4min 27s, sys: 364 ms, total: 4min 28s
Wall time: 4min 29s


In [9]:
df.head()

,sentiment,text,Processed_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",user_mention url aww thats a bummer you shoul...
1,0,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...
2,0,@Kenichan I dived many times for the ball. Man...,user_mention i dived many times for the ball m...
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all....",user_mention no its not behaving at all im mad...


In [10]:
df = df[["Processed_text", "sentiment"]]
df.to_csv("train.csv", index=None)

In [11]:
import torchtext
tweet = torchtext.data.Field(lower=True) # , tokenize="spacy")
targets = torchtext.data.RawField(is_target=True)
fields = [("Processed_text",tweet ), ("sentiment",targets)]

In [12]:
%%time
dataset = TabularDataset(path="./train.csv", format="CSV", fields=fields, skip_header=True)

CPU times: user 38 s, sys: 1.07 s, total: 39.1 s
Wall time: 39.3 s


In [13]:
tweet.build_vocab(dataset, max_size=100_000, min_freq=5)
vocab = tweet.vocab
vocab_size = len(vocab)
print(vocab_size)

57280


In [14]:
train_dataset, valid_dataset = dataset.split(1-TEST_SIZE)

In [15]:
biter = torchtext.data.BucketIterator(dataset=train_dataset, 
                                      batch_size=4,
                                      sort_key=lambda x: len(x.comment_text),
                                      train=True, 
                                      sort=False,
                                      shuffle=True)
for i in biter:
    print(i.Processed_text.shape)
    print(len(i.sentiment))
    break

torch.Size([17, 4])
4


In [16]:
train_biter = torchtext.data.BucketIterator(dataset=train_dataset, 
                                      batch_size=200,
                                      sort_key=lambda x: len(x.comment_text),
                                      train=True, 
                                      sort=False,
                                      shuffle=True)
valid_biter = torchtext.data.BucketIterator(dataset=valid_dataset, 
                                      batch_size=200,
                                      sort_key=lambda x: len(x.comment_text),
                                      train=True, 
                                      sort=False,
                                      shuffle=True)

In [17]:
# MAX = -1
# for text, cls in valid_biter:
#     MAX = max(MAX, torch.max(text).item())
# for text, cls in train_biter:
#     MAX = max(MAX, torch.max(text).item())
# print(MAX)

## Bag of embedding model

In [18]:
import torch.nn as nn

class TextSentiment(nn.Module):
    """
    from torchtext examples
    """
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, 10)
        self.fc2 = nn.Linear(10, num_class)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=0.2)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        r"""
        Arguments:
            text: 1-D tensor representing a bag of text tensors
        """
        x = self.embedding(text)
        x = self.fc(self.drop(x))
        return self.fc2(self.relu(x))

model = TextSentiment(vocab_size, embed_dim=32, num_class=2).to(device)

In [19]:
for text, cls in train_biter:
    cls = torch.tensor([int(i) for i in cls]).to(device)
    text = text.T.to(device)
    output = model(text)
    print(output.shape)
    break

torch.Size([200, 2])


In [20]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.5)

In [21]:
num_epochs = 5

for epoch in range(num_epochs):

    train_loss = 0.0
    valid_loss = 0.0
    valid_acc  = 0.0
    # Train the model
    model.train()
    bar = tq(train_biter, postfix={"train_loss":0.0, "Accuracy":0.0}, leave=False, disable=True)
    for text, cls in bar:
        optimizer.zero_grad()
        cls = torch.tensor([int(i) for i in cls]).to(device)
        text = text.T.to(device)
        output = model(text)
        loss = criterion(output, cls)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        acc = torch.sum(cls == torch.argmax(output, axis=1)).item() / cls.shape[0]
        bar.set_postfix(ordered_dict={"train_loss":loss.item() , "Accuracy":acc})
    
    model.eval()
    with torch.no_grad():
        bar = tq(valid_biter, postfix={"valid_loss":0.0, "Accuracy":0.0}, leave=False, disable=True)
        for text, cls in bar:
            cls = torch.tensor([int(i) for i in cls]).to(device)
            text = text.T.to(device)
            output = model(text)
            loss = criterion(output, cls)
            acc = torch.sum(cls == torch.argmax(output, axis=1)).item() / cls.shape[0]
            valid_loss += loss.item()
            valid_acc += acc
            bar.set_postfix(ordered_dict={"valid_loss":loss.item(), "Accuracy":acc})
    
    print(f"epoch {epoch}")
    print(f"training   loss : {train_loss/len(train_biter)}")
    print(f"validation loss : {valid_loss/len(valid_biter)}")
    print(f"validation acc  : {valid_acc/len(valid_biter)}")
    scheduler.step()

epoch 0
training   loss : 0.4668605595920235
validation loss : 0.4247741750627756
validation acc  : 0.8036156249999997
epoch 1
training   loss : 0.41909288871102035
validation loss : 0.4174092879332602
validation acc  : 0.807590624999999
epoch 2
training   loss : 0.40727002450730654
validation loss : 0.41556562496349214
validation acc  : 0.8093218749999993
epoch 3
training   loss : 0.39925427118781953
validation loss : 0.4162486629374325
validation acc  : 0.8095312499999998
epoch 4
training   loss : 0.3928356837853789
validation loss : 0.4164446508698165
validation acc  : 0.8097031250000011


## LSTM

In [22]:
import torch.nn as nn

class TextSentimentLSTM(nn.Module):
    """
    from torchtext examples
    """
    def __init__(self, vocab_size, embed_dim, num_class, hdim=30):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, sparse=False)
        self.LSTM = nn.LSTM(embed_dim, hdim, 1, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hdim*2, 100)
        self.fc2 = nn.Linear(100, num_class)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=0.2)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        r"""
        Arguments:
            text: 1-D tensor representing a bag of text tensors
        """
        x = self.drop(self.embedding(text))
        x = self.LSTM(x)
        h = torch.transpose(x[1][0], 0, 1)
        h = torch.reshape(h, (h.shape[0], -1))
        h = self.fc(h)
        return self.fc2(self.relu(h))

model = TextSentimentLSTM(vocab_size, embed_dim=100, num_class=2).to(device)
# model.embedding.require_grad = False

In [23]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.5)

In [24]:
num_epochs = 5

for epoch in range(num_epochs):

    train_loss = 0.0
    valid_loss = 0.0
    valid_acc  = 0.0
    # Train the model
    model.train()
    bar = tq(train_biter, postfix={"train_loss":0.0, "Accuracy":0.0}, leave=False, disable=True)
    for text, cls in bar:
        optimizer.zero_grad()
        cls = torch.tensor([int(i) for i in cls]).to(device)
        text = text.T.to(device)
        output = model(text)
        loss = criterion(output, cls)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        acc = torch.sum(cls == torch.argmax(output, axis=1)).item() / cls.shape[0]
        bar.set_postfix(ordered_dict={"train_loss":loss.item() , "Accuracy":acc})
    
    model.eval()
    LOSS = 0.0
    i  = 0
    with torch.no_grad():
        bar = tq(valid_biter, postfix={"valid_loss":0.0, "Accuracy":0.0}, leave=False, disable=True)
        for text, cls in bar:
            cls = torch.tensor([int(i) for i in cls]).to(device)
            text = text.T.to(device)
            output = model(text)
            loss = criterion(output, cls)
            acc = torch.sum(cls == torch.argmax(output, axis=1)).item() / cls.shape[0]
            valid_loss += loss.item()
            valid_acc += acc
            bar.set_postfix(ordered_dict={"valid_loss":loss.item(), "Accuracy":acc})
    
    print(f"epoch {epoch}")
    print(f"training   loss : {train_loss/len(train_biter)}")
    print(f"validation loss : {valid_loss/len(valid_biter)}")
    print(f"validation acc  : {valid_acc/len(valid_biter)}")
    scheduler.step()

epoch 0
training   loss : 0.43034851862117646
validation loss : 0.38796896521002056
validation acc  : 0.8238875000000008
epoch 1
training   loss : 0.37530276467558
validation loss : 0.3734485221654177
validation acc  : 0.8310312500000022
epoch 2
training   loss : 0.351974680935964
validation loss : 0.3708026708196849
validation acc  : 0.8340375000000008
epoch 3
training   loss : 0.3327898846962489
validation loss : 0.376507467860356
validation acc  : 0.8343593750000017
epoch 4
training   loss : 0.31510564665077256
validation loss : 0.3846075169928372
validation acc  : 0.8334687500000031


## Using Pretrained embeddings

In [25]:
import torchtext
tweet = torchtext.data.Field(lower=True) # , tokenize="spacy")
targets = torchtext.data.RawField(is_target=True)
fields = [("Processed_text",tweet ), ("sentiment",targets)]

In [26]:
%%time
dataset = TabularDataset(path="./train.csv", format="CSV", fields=fields, skip_header=True)

CPU times: user 49.4 s, sys: 1.03 s, total: 50.5 s
Wall time: 50.5 s


In [27]:
tweet.build_vocab(dataset, max_size=100_000, min_freq=5, vectors="glove.6B.100d")
vocab = tweet.vocab
vocab_size = len(vocab)
print(vocab_size)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
100%|█████████▉| 398452/400000 [00:23<00:00, 17692.65it/s]

57280


In [28]:
train_dataset, valid_dataset = dataset.split(1-TEST_SIZE)

biter = torchtext.data.BucketIterator(dataset=train_dataset, 
                                      batch_size=4,
                                      sort_key=lambda x: len(x.comment_text),
                                      train=True, 
                                      sort=False,
                                      shuffle=True)

In [29]:
for i in biter:
    print(i.Processed_text.shape)
    print(len(i.sentiment))
    break

torch.Size([17, 4])
4


In [30]:
train_biter = torchtext.data.BucketIterator(dataset=train_dataset, 
                                      batch_size=200,
                                      sort_key=lambda x: len(x.comment_text),
                                      train=True, 
                                      sort=False,
                                      shuffle=True)
valid_biter = torchtext.data.BucketIterator(dataset=valid_dataset, 
                                      batch_size=200,
                                      sort_key=lambda x: len(x.comment_text),
                                      train=True, 
                                      sort=False,
                                      shuffle=True)

In [31]:
import torch.nn as nn

class TextSentiment(nn.Module):
    """
    from torchtext examples
    """
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.embedding.weight.data.copy_(vocab.vectors)
        self.fc = nn.Linear(embed_dim, 10)
        self.fc2 = nn.Linear(10, num_class)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=0.2)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        #self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        r"""
        Arguments:
            text: 1-D tensor representing a bag of text tensors
        """
        x = self.embedding(text)
        x = self.fc(self.drop(x))
        return self.fc2(self.relu(x))

model = TextSentiment(vocab_size, embed_dim=100, num_class=2).to(device)

# for i in model.embedding.parameters():
#     i.requires_grad = False

In [32]:
for text, cls in train_biter:
    cls = torch.tensor([int(i) for i in cls]).to(device)
    text = text.T.to(device)
    output = model(text)
    print(output.shape)
    break

torch.Size([200, 2])


In [33]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.5)

In [34]:
num_epochs = 5

for epoch in range(num_epochs):

    train_loss = 0.0
    valid_loss = 0.0
    valid_acc  = 0.0
    # Train the model
    model.train()
    bar = tq(train_biter, postfix={"train_loss":0.0, "Accuracy":0.0}, leave=False, disable=True)
    for text, cls in bar:
        optimizer.zero_grad()
        cls = torch.tensor([int(i) for i in cls]).to(device)
        text = text.T.to(device)
        output = model(text)
        loss = criterion(output, cls)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        acc = torch.sum(cls == torch.argmax(output, axis=1)).item() / cls.shape[0]
        bar.set_postfix(ordered_dict={"train_loss":loss.item() , "Accuracy":acc})
    
    model.eval()
    LOSS = 0.0
    i  = 0
    with torch.no_grad():
        bar = tq(valid_biter, postfix={"valid_loss":0.0, "Accuracy":0.0}, leave=False, disable=True)
        for text, cls in bar:
            cls = torch.tensor([int(i) for i in cls]).to(device)
            text = text.T.to(device)
            output = model(text)
            loss = criterion(output, cls)
            acc = torch.sum(cls == torch.argmax(output, axis=1)).item() / cls.shape[0]
            valid_loss += loss.item()
            valid_acc += acc
            bar.set_postfix(ordered_dict={"valid_loss":loss.item(), "Accuracy":acc})
    
    print(f"epoch {epoch}")
    print(f"training   loss : {train_loss/len(train_biter)}")
    print(f"validation loss : {valid_loss/len(valid_biter)}")
    print(f"validation acc  : {valid_acc/len(valid_biter)}")
    scheduler.step()

100%|█████████▉| 398452/400000 [00:40<00:00, 17692.65it/s]

epoch 0
training   loss : 0.4581087573617697
validation loss : 0.4188933868892491
validation acc  : 0.8063656250000005
epoch 1
training   loss : 0.41479856812860816
validation loss : 0.41219769479706886
validation acc  : 0.8101218750000008
epoch 2
training   loss : 0.4016960977716371
validation loss : 0.40972853779792784
validation acc  : 0.8119312499999991
epoch 3
training   loss : 0.39214011245407165
validation loss : 0.4098569625988603
validation acc  : 0.8123843750000023
epoch 4
training   loss : 0.38401701556984336
validation loss : 0.4090965586900711
validation acc  : 0.8134250000000016


In [35]:
import torch.nn as nn

class TextSentimentLSTM(nn.Module):
    """
    from torchtext examples
    """
    def __init__(self, vocab_size, embed_dim, num_class, hdim=30):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, sparse=False)
        self.embedding.weight.data.copy_(vocab.vectors)
        self.LSTM = nn.LSTM(embed_dim, hdim, 1, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hdim*2, 100)
        self.fc2 = nn.Linear(100, num_class)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=0.2)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        #self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        r"""
        Arguments:
            text: 1-D tensor representing a bag of text tensors
        """
        x = self.drop(self.embedding(text))
        x = self.LSTM(x)
        h = torch.transpose(x[1][0], 0, 1)
        h = torch.reshape(h, (h.shape[0], -1))
        h = self.fc(h)
        return self.fc2(self.relu(h))

model = TextSentimentLSTM(vocab_size, embed_dim=100, num_class=2).to(device)
# for i in model.embedding.parameters():
#     i.requires_grad = False

In [36]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.5)

In [37]:
num_epochs = 5

for epoch in range(num_epochs):

    train_loss = 0.0
    valid_loss = 0.0
    valid_acc  = 0.0
    # Train the model
    model.train()
    bar = tq(train_biter, postfix={"train_loss":0.0, "Accuracy":0.0}, leave=False, disable=True)
    for text, cls in bar:
        optimizer.zero_grad()
        cls = torch.tensor([int(i) for i in cls]).to(device)
        text = text.T.to(device)
        output = model(text)
        loss = criterion(output, cls)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        acc = torch.sum(cls == torch.argmax(output, axis=1)).item() / cls.shape[0]
        bar.set_postfix(ordered_dict={"train_loss":loss.item() , "Accuracy":acc})
    
    model.eval()
    LOSS = 0.0
    i  = 0
    with torch.no_grad():
        bar = tq(valid_biter, postfix={"valid_loss":0.0, "Accuracy":0.0}, leave=False, disable=True)
        for text, cls in bar:
            cls = torch.tensor([int(i) for i in cls]).to(device)
            text = text.T.to(device)
            output = model(text)
            loss = criterion(output, cls)
            acc = torch.sum(cls == torch.argmax(output, axis=1)).item() / cls.shape[0]
            valid_loss += loss.item()
            valid_acc += acc
            bar.set_postfix(ordered_dict={"valid_loss":loss.item(), "Accuracy":acc})
    
    print(f"epoch {epoch}")
    print(f"training   loss : {train_loss/len(train_biter)}")
    print(f"validation loss : {valid_loss/len(valid_biter)}")
    print(f"validation acc  : {valid_acc/len(valid_biter)}")
    scheduler.step()

epoch 0
training   loss : 0.42490150020457806
validation loss : 0.3847569063864648
validation acc  : 0.8252437500000015
epoch 1
training   loss : 0.37303343913517895
validation loss : 0.373098201174289
validation acc  : 0.8313687500000044
epoch 2
training   loss : 0.3523739409679547
validation loss : 0.3693710339441896
validation acc  : 0.833984375000003
epoch 3
training   loss : 0.33636704393429684
validation loss : 0.37696801419369874
validation acc  : 0.8334687500000014
epoch 4
training   loss : 0.3226567427138798
validation loss : 0.38021334728226064
validation acc  : 0.8331593750000027
